In [1]:
import time
start_time = time.time()
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn import  metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import  cross_val_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso, LogisticRegression)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv',)
submission_df = pd.read_csv('sample_submission.csv')

print('train_df shape: ',train_df.shape)
print('test_df shape: ',test_df.shape)

print("--- %s seconds ---" % round((time.time() - start_time),1))

train_df shape:  (200000, 202)
test_df shape:  (200000, 201)
--- 35.0 seconds ---


In [2]:
start_time = time.time()
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
num_cols = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

std = MinMaxScaler()
scaled = std.fit_transform(train_df[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

#dropping original values merging scaled values for numerical columns
df_data_og = train_df.copy()
data = train_df.drop(columns = num_cols,axis = 1)
data = data.merge(scaled,left_index=True,right_index=True,how = "left")
data = data.drop(columns = ['ID_code'],axis = 1)

std = MinMaxScaler()
scaled = std.fit_transform(test_df[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

#dropping original values merging scaled values for numerical columns
df_test_og = test_df.copy()
test = test_df.drop(columns = num_cols,axis = 1)
test = test.merge(scaled,left_index=True,right_index=True,how = "left")
test = test.drop(columns = ['ID_code'],axis = 1)

seed = 2203
param = {
    'bagging_freq': 5,          
    'bagging_fraction': 0.331,   'boost_from_average':'false',   
    'boost': 'gbdt',             'feature_fraction': 0.0405,     'learning_rate': 0.0123,
    'max_depth': -1,             'metric':'auc',                'min_data_in_leaf': 80,     'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,            'tree_learner': 'serial',   'objective': 'binary',
    'verbosity': 1, 'save_binary': True
}
folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
oof = np.zeros(len(data))
predictions = np.zeros(len(test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(data.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(data.iloc[trn_idx][num_cols], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(data.iloc[val_idx][num_cols], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, 50000, valid_sets = [trn_data, val_data],
                    verbose_eval=2000, early_stopping_rounds = 2000)
    oof[val_idx] = clf.predict(data.iloc[val_idx][num_cols], num_iteration=clf.best_iteration)
    predictions += clf.predict(test[num_cols], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))
#sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
#sub["target"] = predictions
#sub.to_csv("submission.csv", index=False)

print("--- %s seconds ---" % round((time.time() - start_time),1))

Fold 0
Training until validation scores don't improve for 2000 rounds.
[2000]	training's auc: 0.915744	valid_1's auc: 0.892776
[4000]	training's auc: 0.928436	valid_1's auc: 0.898115
[6000]	training's auc: 0.937753	valid_1's auc: 0.899723
[8000]	training's auc: 0.945783	valid_1's auc: 0.899943
Early stopping, best iteration is:
[6865]	training's auc: 0.941314	valid_1's auc: 0.900069
Fold 1
Training until validation scores don't improve for 2000 rounds.
[2000]	training's auc: 0.915765	valid_1's auc: 0.893937
[4000]	training's auc: 0.928437	valid_1's auc: 0.898435
[6000]	training's auc: 0.937718	valid_1's auc: 0.899521
[8000]	training's auc: 0.945919	valid_1's auc: 0.899591
Early stopping, best iteration is:
[7274]	training's auc: 0.943054	valid_1's auc: 0.899809
Fold 2
Training until validation scores don't improve for 2000 rounds.
[2000]	training's auc: 0.916142	valid_1's auc: 0.887191
[4000]	training's auc: 0.928808	valid_1's auc: 0.893296
[6000]	training's auc: 0.938014	valid_1's auc

KeyboardInterrupt: 

In [ ]:
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))
#sub = pd.DataFrame()
#sub["target"] = predictions
#sub.to_csv("submission.csv", index=False)

#sub['target']= pd.DataFrame(ABC.predict_proba(test)[:,1])
#submission_df.to_csv('submission_seed.csv', index=False)
#gathering_df.drop(['target'])
#gathering_df.to_csv('gathering_df.csv',index = False)